In [ ]:

!pip install transformers accelerate gradio


In [ ]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr

model_name = "HuggingFaceH4/zephyr-7b-alpha"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


In [ ]:
def chat_with_admission_bot(user_input, history=[]):
    system_instruction = (
        "You are a helpful and friendly university admissions assistant. "
        "Provide concise, polite, and informative answers related to university "
        "applications, courses, fees, scholarships, deadlines, and the admissions process."
    )
    
    full_prompt = f"""{system_instruction}
User: {user_input}
Assistant:"""

    inputs = tokenizer(full_prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(inputs, max_new_tokens=200, temperature=0.7, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response.split("Assistant:")[-1].strip()

In [ ]:

interface = gr.Interface(
    fn=chat_with_admission_bot,
    inputs="text",
    outputs="text",
    title="University Admission Assistant",
    description="Ask anything about university applications, courses, tuition fees, deadlines, scholarships, and more.",
)

interface.launch()
